Evidencia de aprendizaje (EA2). Taller: procesamiento de datos en una infraestructura cloud

Presentado por Diego Barros Ballestas

**Diseño el esquema que almacenará los datos**

_Descripción las entidades y campos clave del dataset (tipos de datos, llaves, nulabilidad)._


**Descripción del dataset**: Ventas y/o comercio de laptops

**Entidades principales**

📊 Entidad de Hecho
**Sale** Registra cada transacción de venta asociada a un modelo
| Campo                | Tipo de dato       | Nulabilidad | Descripción                  |
| -------------------- | ------------------ | ----------- | ---------------------------- |
| **sale_id**          | SERIAL PRIMARY KEY | NOT NULL    | Identificador de venta.      |
| **laptop_model_id**  | INT                | NOT NULL    | Llave foránea → LaptopModel. |
| **sale_date**        | DATE               | NOT NULL    | Fecha de la venta.           |
| **price_sold**       | DECIMAL(10,2)      | NOT NULL    | Precio al que se vendió.     |
| **customer_segment** | VARCHAR(100)       | NULL        | Segmento del cliente.        |
| **location**         | VARCHAR(100)       | NULL        | Ciudad/país de la venta.     |

📊 Entidad de Hecho
**Inventory**
Controla el stock disponible por ubicación
| Campo                | Tipo de dato       | Nulabilidad | Descripción                  |
| -------------------- | ------------------ | ----------- | ---------------------------- |
| **inventory_id**     | SERIAL PRIMARY KEY | NOT NULL    | Identificador de inventario. |
| **laptop_model_id**  | INT                | NOT NULL    | Llave foránea → LaptopModel. |
| **store_location**   | VARCHAR(120)       | NOT NULL    | Lugar donde se almacena.     |
| **quantity_on_hand** | INT                | NOT NULL    | Cantidad disponible.         |
| **date_recorded**    | DATE               | NOT NULL    | Fecha del registro.          |




📦 **Entidad de Dimension:** 
**LaptopModel**

Almacena la información técnica de cada modelo de laptop.

| Campo               | Tipo de dato       | Nulabilidad | Descripción                                      |
| ------------------- | ------------------ | ----------- | ------------------------------------------------ |
| **laptop_model_id** | SERIAL PRIMARY KEY | NOT NULL    | Identificador único del modelo.                  |
| **brand**           | VARCHAR(100)       | NOT NULL    | Marca del fabricante.                            |
| **model_name**      | VARCHAR(150)       | NOT NULL    | Nombre o referencia del modelo.                  |
| **processor**       | VARCHAR(100)       | NOT NULL    | Modelo de procesador.                            |
| **ram_gb**          | INT                | NOT NULL    | Cantidad de RAM.                                 |
| **storage_gb**      | INT                | NOT NULL    | Capacidad de almacenamiento.                     |
| **storage_type**    | VARCHAR(50)        | NOT NULL    | SSD o HDD.                                       |
| **gpu**             | VARCHAR(100)       | NULL        | Tarjeta gráfica. Puede ser nula si es integrada. |
| **screen_inches**   | DECIMAL(3,1)       | NOT NULL    | Tamaño de pantalla.                              |
| **release_year**    | INT                | NULL        | Año de lanzamiento (puede faltar).               |

🏷️ **Entidad de Dimension:**
**Promotion**

Almacena los descuentos y promociones aplicadas a ciertos modelos.

| Campo                | Tipo de dato       | Nulabilidad | Descripción                  |
| -------------------- | ------------------ | ----------- | ---------------------------- |
| **promotion_id**     | SERIAL PRIMARY KEY | NOT NULL    | Identificador de promoción.  |
| **laptop_model_id**  | INT                | NOT NULL    | Llave foránea → LaptopModel. |
| **promo_start_date** | DATE               | NOT NULL    | Inicio de la promo.          |
| **promo_end_date**   | DATE               | NOT NULL    | Fin de la promo.             |
| **discount_rate**    | DECIMAL(5,2)       | NOT NULL    | % de descuento.              |
| **notes**            | VARCHAR(255)       | NULL        | Comentarios opcionales.      |




**DLL spark con represtancion del esquema**

In [0]:
%sql

CREATE TABLE IF NOT EXISTS laptop_model (
    laptop_model_id     INT,
    brand               STRING NOT NULL,
    model_name          STRING NOT NULL,
    processor           STRING NOT NULL,
    ram_gb              INT NOT NULL,
    storage_gb          INT NOT NULL,
    storage_type        STRING NOT NULL,
    gpu                 STRING,
    screen_inches       DECIMAL(3,1) NOT NULL,
    release_year        INT
)
USING DELTA
COMMENT "Catálogo de modelos de laptop. PK: laptop_model_id";


CREATE TABLE IF NOT EXISTS sale (
    sale_id             INT,
    laptop_model_id     INT NOT NULL,
    sale_date           DATE NOT NULL,
    price_sold          DECIMAL(10,2) NOT NULL,
    customer_segment    STRING,
    location            STRING
)
USING DELTA
COMMENT "Registro de ventas de laptops. PK: sale_id. FK: laptop_model_id -> laptop_model";

CREATE TABLE IF NOT EXISTS inventory (
    inventory_id        INT,
    laptop_model_id     INT NOT NULL,
    store_location      STRING NOT NULL,
    quantity_on_hand    INT NOT NULL,
    date_recorded       DATE NOT NULL
)
USING DELTA
COMMENT "Inventario por modelo y ubicación. PK: inventory_id. FK: laptop_model_id -> laptop_model";

CREATE TABLE IF NOT EXISTS promotion (
    promotion_id        INT,
    laptop_model_id     INT NOT NULL,
    promo_start_date    DATE NOT NULL,
    promo_end_date      DATE NOT NULL,
    discount_rate       DECIMAL(5,2) NOT NULL,
    notes               STRING
)
USING DELTA
COMMENT "Promociones aplicadas a modelos. PK: promotion_id. FK: laptop_model_id -> laptop_model";


**Diccionario de datos**

| **Tabla**       | **Campo**        | **Tipo**      | **Nulo** | **Descripción**                        |
| --------------- | ---------------- | ------------- | -------- | -------------------------------------- |
| **LaptopModel** | laptop_model_id  | INT (PK)      | NO       | Identificador único del modelo.        |
|                 | brand            | VARCHAR(100)  | NO       | Marca de la laptop.                    |
|                 | model_name       | VARCHAR(150)  | NO       | Nombre del modelo.                     |
|                 | processor        | VARCHAR(100)  | NO       | Procesador instalado.                  |
|                 | ram_gb           | INT           | NO       | Capacidad de RAM.                      |
|                 | storage_gb       | INT           | NO       | Tamaño de almacenamiento.              |
|                 | storage_type     | VARCHAR(50)   | NO       | Tipo de almacenamiento (SSD/HDD).      |
|                 | gpu              | VARCHAR(100)  | SÍ       | Tarjeta gráfica (puede ser integrada). |
|                 | screen_inches    | DECIMAL(3,1)  | NO       | Tamaño de pantalla.                    |
|                 | release_year     | INT           | SÍ       | Año de lanzamiento.                    |
| **Sale**        | sale_id          | INT (PK)      | NO       | Identificador de venta.                |
|                 | laptop_model_id  | INT (FK)      | NO       | Modelo vendido.                        |
|                 | sale_date        | DATE          | NO       | Fecha de venta.                        |
|                 | price_sold       | DECIMAL(10,2) | NO       | Precio final vendido.                  |
|                 | customer_segment | VARCHAR(100)  | SÍ       | Tipo de cliente.                       |
|                 | location         | VARCHAR(100)  | SÍ       | Ciudad/país de venta.                  |
| **Inventory**   | inventory_id     | INT (PK)      | NO       | Registro del inventario.               |
|                 | laptop_model_id  | INT (FK)      | NO       | Modelo asociado.                       |
|                 | store_location   | VARCHAR(120)  | NO       | Ubicación del inventario.              |
|                 | quantity_on_hand | INT           | NO       | Cantidad disponible.                   |
|                 | date_recorded    | DATE          | NO       | Fecha de registro.                     |
| **Promotion**   | promotion_id     | INT (PK)      | NO       | Identificador de la promoción.         |
|                 | laptop_model_id  | INT (FK)      | NO       | Modelo al que aplica.                  |
|                 | promo_start_date | DATE          | NO       | Inicio de promoción.                   |
|                 | promo_end_date   | DATE          | NO       | Fin de promoción.                      |
|                 | discount_rate    | DECIMAL(5,2)  | NO       | Porcentaje de descuento.               |
|                 | notes            | VARCHAR(255)  | SÍ       | Comentarios adicionales.               |


**Configuracion y evidencia la infraestructura en Databricks CE**

Crear y Configurar un Cluster

En la barra lateral izquierda de tu espacio de trabajo de Databricks, haz clic en el icono de "Compute".
Haz clic en "Create Cluster".
Configura:
Cluster Name: Actividad2
Databricks Runtime Version: e.g., 13.0 LTS (Scala 2.12, Spark 3.4.0)
Python Version: e.g., 3.12.3
Cluster Mode: Standard
Autoscaling: Activado (min 1 nodo – max 4 nodos)
Worker Type: e.g., 4 vCPU / 16 GB RAM
Haz clic en Create Cluster.

 - Esto creará un clúster escalable listo para ejecutar Spark y SQL.

 - Como no es posible realizarlo en la versión Free de Databricks estos serían los resultados.

Configuración del Clúster

Nombre del clúster: Actividad2
Databricks Runtime: 13.0 LTS
Python: 3.12.3
Modo: Standard
Núcleos/RAM: 4 vCPU / 16 GB RAM
Autoscaling: 1–4 nodos
b. Version de Python y Spark
Configuración del SparkContext

Si ejecutáramos el siguiente código:

python for item in spark.sparkContext.getConf().getAll(): print(item)

Esto imprimirá: versión de Spark, configuración de Python, directorios, núcleos y RAM asignados.

Como no es posible en la versión free, solo mostraremos la versión de Spark y Python.

Versión de Spark
Se imprime usando spark.version

Fuente: https://www.databricks.com/blog/2020/07/02/allow-simple-cluster-creation-with-full-admin-control-using-cluster-policies.html



In [0]:
# Versión de Spark
print("Versión de Spark:", spark.version)

Versión de Spark: 4.0.0


**Obtencion de los datos de Kaggle y creacion de una tabla**

In [0]:
!pip install kagglehub[pandas-datasets]>=0.3.8

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Importamos las librerias necesarias

In [0]:

import os
import zipfile
import kagglehub
import pandas as pd 

Extracción de datos

In [0]:
def download_dataset_zip(url = ""):
        print("Descargando dataset desde Kaggle...")
        dataset_path = kagglehub.dataset_download(url)
        print("Ruta al dataset:", dataset_path)
        return dataset_path
    
def extract_zip_files(dataset_path):
        zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
        if zip_files:
            zip_file = os.path.join(dataset_path, zip_files[0])
            extract_dir = os.path.join(dataset_path, "extracted")
            os.makedirs(extract_dir, exist_ok=True)
            print(f"Extrayendo {zip_file} en {extract_dir}...")
            with zipfile.ZipFile(zip_file, "r") as z:
                z.extractall(extract_dir)
            return extract_dir
        else:
            # Si no se encuentra un ZIP, se verifica si existen archivos CSV en la ruta
            csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
            if csv_files:
                print("No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.")
                return dataset_path
            else:
                raise FileNotFoundError("No se encontró ningún archivo .zip ni archivos .csv en la ruta del dataset")

def create_csv(csv_dir, csv_name=None):
    if csv_name:
        file_path = os.path.join(csv_dir, csv_name)
        print(f"Leyendo {file_path}...")
        df = pd.read_csv(file_path, encoding="latin1")
        print("CSV creado correctamente")
        return df
    else:
        csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("No se encontraron archivos CSV en el directorio extraído")
        for file in csv_files:
            file_path = os.path.join(csv_dir, file)
            print(f"Leyendo {file_path}...")
            df = pd.read_csv(file_path, encoding="latin1")
        print("CSV creado correctamente")
        return df

Descarga del dataset

In [0]:
df = pd.DataFrame()
dataset_path = download_dataset_zip("siddiquifaiznaeem/laptop-sales-price-prediction-dataset-2024") 
csv_dir = extract_zip_files(dataset_path)
df = create_csv(csv_dir, csv_name="laptop_cleaned2.csv")

Descargando dataset desde Kaggle...


100%|██████████| 36.7k/36.7k [00:00<00:00, 3.34MB/s]

Extracting files...
Ruta al dataset: /home/spark-a1e8af6d-16a4-46af-95fd-ab/.cache/kagglehub/datasets/siddiquifaiznaeem/laptop-sales-price-prediction-dataset-2024/versions/1
No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.
Leyendo /home/spark-a1e8af6d-16a4-46af-95fd-ab/.cache/kagglehub/datasets/siddiquifaiznaeem/laptop-sales-price-prediction-dataset-2024/versions/1/laptop_cleaned2.csv...
CSV creado correctamente


In [0]:
df.head(3)

,Unnamed: 0,Name,Brand,Price,Rating,Processor_brand,Processor_name,Processor_variant,Processor_gen,Core_per_processor,Total_processor,Execution_units,Low_Power_Cores,Energy_Efficient_Units,Threads,RAM_GB,RAM_type,Storage_capacity_GB,Storage_type,Graphics_name,Graphics_brand,Graphics_GB,Graphics_integreted,Display_size_inches,Horizontal_pixel,Vertical_pixel,ppi,Touch_screen,Operating_system
0,0,HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen...,HP,50399,4.30,AMD,AMD Ryzen 5,5600H,5.0,6.0,NaN,NaN,0.0,0,12.0,8,DDR4,512,SSD,AMD Radeon RX 6500M,AMD,4.0,False,15.6,1920,1080,141.21,True,Windows 11 OS
1,1,Lenovo V15 G4 â82YU00W7IN Laptop (AMD Ryzen ...,Lenovo,26690,4.45,AMD,AMD Ryzen 3,7320U,7.0,4.0,NaN,NaN,0.0,0,8.0,8,LPDDR5,512,SSD,AMD Radeon Graphics,AMD,NaN,False,15.6,1920,1080,141.21,False,Windows 11 OS
2,2,HP 15s-fq5007TU Laptop (12th Gen Core i3/ 8GB/...,HP,37012,4.65,Intel,Intel Core i3,1215U,12.0,6.0,2.0,4.0,0.0,0,8.0,8,DDR4,512,SSD,Intel UHD Graphics,Intel,NaN,False,15.6,1920,1080,141.21,False,Windows 11 OS


Creacion del catalgo y esquema

In [0]:
%sql
-- 1. Crea el Catálogo principal (si no existe)
CREATE CATALOG IF NOT EXISTS azcomputers;

-- 2. Crea el Esquema de Ventas (base de datos)
CREATE SCHEMA IF NOT EXISTS azcomputers.sale_schema;

-- 3. Crea el Volume para almacenar archivos no tabulares,

CREATE VOLUME IF NOT EXISTS azcomputers.sale_schema.vol_sales;

In [0]:

spark_df = spark.createDataFrame(df)

Creacion de la tabla en Spark

In [0]:
spark_df = spark_df.withColumnRenamed('Unnamed: 0', 'Unnamed_0'); spark_df.write.mode("overwrite").saveAsTable("azcomputers.sale_schema.tbl_datesfrom_spk")

Descripcion de la tabla

In [0]:
%sql
DESCRIBE TABLE azcomputers.sale_schema.tbl_datesfrom_spk;

col_name,data_type,comment
Unnamed_0,bigint,null
Name,string,null
Brand,string,null
Price,bigint,null
Rating,double,null
Processor_brand,string,null
Processor_name,string,null
Processor_variant,string,null
Processor_gen,double,null
Core_per_processor,double,null


Descripcion de los datos

In [0]:

# 1. Cargamos la tabla a un DataFrame
df_datessales = spark.table("azcomputers.sale_schema.tbl_datesfrom_spk")

# 2. Seleccionamos solo las columnas de tipo numérico (double, int, long)
#    y excluimos las de tipo string y timestamp.
numeric_cols = [
    f.name for f in df_datessales.schema
    if f.dataType.typeName() in ('double', 'decimal', 'float', 'integer', 'long')
]

# 3. Creamos un nuevo DataFrame solo con esas columnas y aplicamos describe()
df_numeric_stats = df_datessales.select(*numeric_cols)
display(df_numeric_stats.describe())

summary,Unnamed_0,Price,Rating,Processor_gen,Core_per_processor,Total_processor,Execution_units,Low_Power_Cores,Energy_Efficient_Units,Threads,RAM_GB,Storage_capacity_GB,Graphics_GB,Display_size_inches,Horizontal_pixel,Vertical_pixel,ppi
count,1020,1020,1020,891,1008,573,573,1020,1020,972,1020,1020,368,1020,1020,1020,1020
mean,509.5,82063.47450980393,4.3736764705882365,10.450056116722783,8.572420634920634,3.926701570680628,6.99825479930192,0.08627450980392157,0.043137254901960784,12.817901234567902,13.992156862745098,627.7333333333333,5.940217391304348,15.16377450980403,2035.5127450980392,1214.0196078431372,157.17826470588287
stddev,294.5929394944828,66502.15060661969,0.23329510858395427,2.966579291621357,4.375011793844324,1.954429014487957,2.680217040188408,0.40653137120951016,0.20326568560475508,5.677459045534028,7.189563997619087,316.91167891735205,2.66713001443877,1.0015365859568317,409.2092890342419,306.86308590559287,33.58571285497695
min,0,8000,3.95,1.0,2.0,1.0,4.0,0.0,0,2.0,2,32,2.0,11.6,1080,768,100.45
max,1019,599990,4.75,14.0,24.0,12.0,16.0,2.0,1,32.0,64,4000,16.0,18.0,3840,2560,337.93


Validacion de las tablas creadas

In [0]:
%sql
SHOW TABLES IN azcomputers.sale_schema;

database,tableName,isTemporary
sale_schema,tbl_datesfrom_spk,false


Ingesta de datos en Keagle

In [0]:
%sql
LIST '/Volumes/azcomputers/sale_schema/vol_sales';

path,name,size,modification_time
/Volumes/azcomputers/sale_schema/vol_sales/laptop_cleaned2.csv,laptop_cleaned2.csv,257533,1763957892000


Carga de datos

In [0]:
# 1. Lee la data directamente desde el Volume con Spark (sin crear la tabla)
rut_csv_volume = 'dbfs:/Volumes/azcomputers/sale_schema/vol_sales/laptop_cleaned2.csv'

df_diagnostico = spark.read.csv(
  rut_csv_volume,
  header=True,
  inferSchema=True
)

# 2. Imprime los nombres de las columnas que Spark REALMENTE leyó
print(df_diagnostico.columns)

['_c0', 'Name', 'Brand', 'Price', 'Rating', 'Processor_brand', 'Processor_name', 'Processor_variant', 'Processor_gen', 'Core_per_processor', 'Total_processor', 'Execution_units', 'Low_Power_Cores', 'Energy_Efficient_Units', 'Threads', 'RAM_GB', 'RAM_type', 'Storage_capacity_GB', 'Storage_type', 'Graphics_name', 'Graphics_brand', 'Graphics_GB', 'Graphics_integreted', 'Display_size_inches', 'Horizontal_pixel', 'Vertical_pixel', 'ppi', 'Touch_screen', 'Operating_system']


Creacion de vista temporal

In [0]:
%sql

CREATE OR REPLACE TEMPORARY VIEW raw_csv_view 
USING CSV
OPTIONS (
  'path' = '/Volumes/azcomputers/sale_schema/vol_sales/laptop_cleaned2.csv',
  'header' = 'true',
  'inferSchema' = 'true',
  'timestampFormat' = 'yyyy-MM-dd HH:mm:ss'
);

DESCRIBE raw_csv_view;
     

col_name,data_type,comment
_c0,int,null
Name,string,null
Brand,string,null
Price,int,null
Rating,double,null
Processor_brand,string,null
Processor_name,string,null
Processor_variant,string,null
Processor_gen,double,null
Core_per_processor,double,null


Creacion de la tabla con SQL

In [0]:
%sql
CREATE TABLE IF NOT EXISTS azcomputers.sale_schema.tbl_datesfrom
AS SELECT 
 _c0,
 Name,
 Brand,
 Price,
 Rating,
 Processor_brand,
 Processor_name,
 Processor_variant,
 Processor_gen,
 Core_per_processor,
 Total_processor,
 Execution_units,
 Low_Power_Cores,
 Energy_Efficient_Units,
 Threads,
 RAM_GB,
 RAM_type,
 Storage_capacity_GB,
 Storage_type,
 Graphics_name,
 Graphics_brand,
 Graphics_GB,
 Graphics_integreted,
 Display_size_inches,
 Horizontal_pixel,
 Vertical_pixel,
 ppi,
 Touch_screen,
 Operating_system
FROM raw_csv_view;

num_affected_rows,num_inserted_rows


Cantidad de filas que se cargaron

In [0]:
%sql
SELECT COUNT(*) FROM azcomputers.sale_schema.tbl_datesfrom;

COUNT(*)
1020


Descripcion de detalle

In [0]:
%sql
DESCRIBE DETAIL azcomputers.sale_schema.tbl_datesfrom;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,013f592d-a6f8-42cc-9347-92d1ff2cfd5a,azcomputers.sale_schema.tbl_datesfrom,null,,2025-11-24T04:28:33.258Z,2025-11-24T04:28:35.000Z,List(),List(),1,43909,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


**Validaciones en Spark Y SQL**

In [0]:
%sql
DESCRIBE TABLE azcomputers.sale_schema.tbl_datesfrom;

col_name,data_type,comment
_c0,int,null
Name,string,null
Brand,string,null
Price,int,null
Rating,double,null
Processor_brand,string,null
Processor_name,string,null
Processor_variant,string,null
Processor_gen,double,null
Core_per_processor,double,null


In [0]:
%sql
SHOW CREATE TABLE azcomputers.sale_schema.tbl_datesfrom;

createtab_stmt
"CREATE TABLE azcomputers.sale_schema.tbl_datesfrom ( _c0 INT, Name STRING, Brand STRING, Price INT, Rating DOUBLE, Processor_brand STRING, Processor_name STRING, Processor_variant STRING, Processor_gen DOUBLE, Core_per_processor DOUBLE, Total_processor DOUBLE, Execution_units DOUBLE, Low_Power_Cores DOUBLE, Energy_Efficient_Units INT, Threads DOUBLE, RAM_GB INT, RAM_type STRING, Storage_capacity_GB INT, Storage_type STRING, Graphics_name STRING, Graphics_brand STRING, Graphics_GB DOUBLE, Graphics_integreted BOOLEAN, Display_size_inches DOUBLE, Horizontal_pixel INT, Vertical_pixel INT, ppi DOUBLE, Touch_screen BOOLEAN, Operating_system STRING) USING delta COLLATION 'UTF8_BINARY' TBLPROPERTIES ( 'delta.enableDeletionVectors' = 'true', 'delta.feature.appendOnly' = 'supported', 'delta.feature.deletionVectors' = 'supported', 'delta.feature.invariants' = 'supported', 'delta.minReaderVersion' = '3', 'delta.minWriterVersion' = '7', 'delta.parquet.compression.codec' = 'zstd')"


In [0]:
spark_df.printSchema()

root
 |-- Unnamed_0: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Price: long (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Processor_brand: string (nullable = true)
 |-- Processor_name: string (nullable = true)
 |-- Processor_variant: string (nullable = true)
 |-- Processor_gen: double (nullable = true)
 |-- Core_per_processor: double (nullable = true)
 |-- Total_processor: double (nullable = true)
 |-- Execution_units: double (nullable = true)
 |-- Low_Power_Cores: double (nullable = true)
 |-- Energy_Efficient_Units: long (nullable = true)
 |-- Threads: double (nullable = true)
 |-- RAM_GB: long (nullable = true)
 |-- RAM_type: string (nullable = true)
 |-- Storage_capacity_GB: long (nullable = true)
 |-- Storage_type: string (nullable = true)
 |-- Graphics_name: string (nullable = true)
 |-- Graphics_brand: string (nullable = true)
 |-- Graphics_GB: double (nullable = true)
 |-- Graphics_integreted: boolean (nul

**Vetajas y desventajas de SQL y Spark**

| Aspecto                | SQL       | Spark |
| --------------- | ---------------- | ------------- |
| Escalabilidad              | Diseñado para un solo servidor o clúster limitado.      | Escala horizontalmente en miles de nodos para big data. |
| Volumen de datos              | Ideal para datos medianos/pequeños en discos locales.     | Diseñado para big data (TB–PB) y procesamiento distribuido. |
| Procesamiento             | OLTP/OLAP clásico; transacciones ACID nativas.     | Procesamiento distribuido, batch, streaming, ML; ACID depende del sistema de almacenamiento. |
| Almacenamiento           | Necesariamente Requiere una base de datos relacional     | No impone almacenamiento: puede leer de CSV, Parquet, HDFS, S3, Delta Lake, etc.|
| Lenguaje          | Se dificulta un poco por el manejo del lenguaje aprender mucho mas    | Se dificulta un poco por el manejo del lenguaje aprender mucho mas|